In [2]:
import sys
import os
import collections
from scipy import stats

In [3]:
folder = 'Z:\celegans\secretome'

In [4]:
with open('top51.fasta') as f:
    top57 = f.readlines()
top57 = [t for t in top57 if t.startswith('>')]
top57 = [t.split('|')[1] for t in top57 if t.startswith('>')]

In [51]:
len(top57)

57

In [5]:
files = [f for f in os.listdir(folder) if f.endswith('.fa')]

In [6]:
len(files)

3672

In [7]:
aa = set()

In [8]:
def count_aa(file, top57='', ignore_length=-1):
    with open(file, 'r') as f:
        lines = f.readlines()
    file = file.split('.')[0].rsplit('\\')[-1]
    if file in top57:
        return
        #top57.pop(top57.index(file))
    counts = collections.defaultdict(int)
    counter = 0
    for line in lines:
        if line.startswith('>'):
            continue
        for x in line.strip():
            counter += 1
            if counter < ignore_length:
                continue
            counts[x] += 1
            aa.add(x)
    s = sum(counts.values())
    for k, v in counts.items():
        counts[k] = v /s
    return counts

In [9]:
counts = {}

for file in files:
    c = count_aa(os.path.join(folder, file), top57=top57)
    if c is not None:
        counts[file.split('.')[0]] = c

In [10]:
all_proteins = [f for f in os.listdir(os.path.abspath(os.path.join(folder, '..'))) if f.endswith('fa')]

In [11]:
counts_all = {}

for file in all_proteins:
    c = count_aa(os.path.join('z:\\celegans', file), top57=top57)
    if c is not None:
        counts_all[file.split('.')[0]] = c

In [12]:
len(counts)

3615

In [13]:
counts_top = {}

for file in top57:
    c = count_aa(os.path.join(folder, file + '.fa'), top57=[])
    if c is not None:
        counts_top[file] = c

In [14]:
len(files)

3672

In [53]:
for a in aa:
    prob1 = []
    prob2 = []
    for prob in counts.values():
        prob1.append(prob[a])
    for prob in counts_top.values():
        prob2.append(prob[a])
    ttest = stats.ttest_ind(prob1, prob2)
    if ttest[1] < 0.05:
        print(a, ttest[1])
        print(sum(prob1)/len(prob1) * 100, sum(prob2)/len(prob2) * 100)

N 0.022319619428223518
5.1609674790179065 5.860823181368706


In [54]:
len(counts)

3615

In [16]:
sum(prob1)/len(prob1), sum(prob2)/len(prob2)

(0.03613876118319215, 0.0385762822089534)

In [17]:
for a in aa:
    prob1 = []
    prob2 = []
    for prob in counts.values():
        prob1.append(prob[a])
    for prob in counts_all.values():
        prob2.append(prob[a])
    ttest = stats.ttest_ind(prob1, prob2)
    if ttest[1] < 0.05:
        print(a, ttest)
        print(sum(prob1)/len(prob1), sum(prob2)/len(prob2))

V Ttest_indResult(statistic=-4.61249925971557, pvalue=3.994288404368956e-06)
0.06018687518794115 0.06184569611567519
W Ttest_indResult(statistic=2.5572666489982363, pvalue=0.010554500309739954)
0.011596430896333762 0.011177800231394513
H Ttest_indResult(statistic=-12.77156676533676, pvalue=2.928208237269525e-37)
0.019720967836068505 0.02297354521137021
S Ttest_indResult(statistic=-6.393134458889781, pvalue=1.6480555415656495e-10)
0.07687435725086472 0.08012224585308904
R Ttest_indResult(statistic=-18.071306236441057, pvalue=1.2556070649816506e-72)
0.04445840514490193 0.05213798868721801
T Ttest_indResult(statistic=10.930310889157514, pvalue=9.267894598423227e-28)
0.06304535868549616 0.05818017467723801
K Ttest_indResult(statistic=-6.7987820099063505, pvalue=1.0739901957946162e-11)
0.05950816800736238 0.063163407839982
I Ttest_indResult(statistic=-12.362692867322785, pvalue=5.021739694596554e-35)
0.05637527933013214 0.06177986939882165
A Ttest_indResult(statistic=4.6571441978732055, pva

In [19]:
import pandas as pd
import pickle
import numpy as np
import scipy.spatial
import sklearn.cluster

In [20]:
with open(os.path.abspath(os.path.join(os.getcwd(), r'../secondary-structure-parser', 'df_all.pickle')), 'rb') as f:
    df = pickle.load(f)

In [21]:
points = []
for t in sorted(top57):
    x = sum(df.loc[t]['probabilities_0']) / len(df.loc[t])
    y = sum(df.loc[t]['probabilities_1']) / len(df.loc[t])
    z = sum(df.loc[t]['disordered']) / len(df.loc[t])
    points.append([x, y, z])

In [37]:
points = np.array(points)
dist_matrix = scipy.spatial.distance_matrix(points, points)

clusters = sklearn.cluster.KMeans(random_state=42).fit_predict(dist_matrix)

In [38]:
composition_cluster = {}
proteins_in_cluster = {}
names = sorted(top57)
for i in range(max(clusters) + 1):
    composition_cluster[i] = {}
    proteins_in_cluster[i] = []
    for ix, cluster in enumerate(clusters):
        if cluster == i:
            proteins_in_cluster[i].append(names[ix])
    for protein in proteins_in_cluster[i]:
        c = count_aa(os.path.join(folder, protein + '.fa'))
        composition_cluster[i][protein] = c

In [39]:
for i in composition_cluster:
    for a in aa:
        prob1 = []
        prob2 = []
        for prob in counts.values():
            prob1.append(prob[a])
        for prob in composition_cluster[i].values():
            prob2.append(prob[a])
        ttest = stats.ttest_ind(prob1, prob2)
        if ttest[1] < 0.01:
            print('AA: {}, Cluster: {}, p-value: {:.6f}'.format(a, i, ttest[1]))
            print(sum(prob1)/len(prob1), sum(prob2)/len(prob2))

AA: N, Cluster: 0, p-value: 0.005621
0.05160967479017906 0.07563428684499778
AA: W, Cluster: 1, p-value: 0.000707
0.011596430896333762 0.020149641741204417
AA: H, Cluster: 4, p-value: 0.003116
0.019720967836068505 0.04106608060933408
AA: I, Cluster: 4, p-value: 0.001688
0.05637527933013214 0.020417078682329545
AA: G, Cluster: 4, p-value: 0.000000
0.06806909135451036 0.22006571306030426
AA: L, Cluster: 4, p-value: 0.003225
0.0789808631018571 0.0382549450536469
AA: P, Cluster: 4, p-value: 0.000001
0.05397362097293895 0.13763043740478836


C:\Users\ashafix\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\ashafix\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\ashafix\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\ashafix\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\ashafix\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
proteins_in_cluster[7]

['O46001', 'O61952', 'O62455', 'P91047', 'Q22903']

In [150]:
sum(df.loc[t]['disordered']) / len(df.loc[t])

0.12956810631229235

In [147]:
df.loc[t]

,prediction,probabilities_0,probabilities_1,probabilities_2,disordered,probability
1,C,0.003,0.000,0.996,True,0.93
2,H,0.808,0.001,0.191,True,0.94
3,H,0.970,0.001,0.030,True,0.95
4,H,0.982,0.000,0.018,True,0.96
5,H,0.992,0.000,0.008,True,0.95
6,H,0.997,0.000,0.003,True,0.96
7,H,0.996,0.000,0.004,True,0.96
8,H,0.994,0.000,0.006,True,0.98
9,H,0.992,0.000,0.008,True,0.98
10,H,0.993,0.000,0.007,True,0.98
